In [1]:
import numpy as np
import scipy
from scipy.linalg import solve_triangular
import pdb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.random.seed(12)

## (a)

In [2]:
def normalize(x):
    
    return x / norm(x)

def norm(x):
    
    return np.sqrt((x**2).sum())

def GS_QR(X):
    A = X.copy()
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    Q[:, 0] = normalize(A[:, 0])
    R[0,0] = norm(A[:, 0])
    for i in range(1, n):
        R[0, i] = (Q[:, 0] * A[:, i]).sum()
        A[:, i] -= R[0, i] * Q[:, 0] 
    
    for j in range(1, n):
        Q[:, j] = normalize(A[:, j])
        R[j,j] = norm(A[:, j])
        for i in range(j + 1, n):
            R[j, i] = (Q[:, j] * A[:, i]).sum()
            A[:, i] -= R[j, i] * Q[:, j] 
        
    return Q, R



## (b)

In [3]:
def house(x):
    a = - np.sign(x[0]) * norm(x)
    v0 = np.sqrt(0.5 * (1 - x[0] / a))
    v = - x / (2 * a * v0)
    v[0] = v0
    
    return v, a

def reflection(x, v):
    
    return x - 2 * np.sum(x * v) * v

def Reflection_matrix(X,v):## todo: speed up
    HX = np.zeros(shape = X.shape)
    for i in range(X.shape[1]):
        HX[:, i] = reflection(X[:, i], v)
        
    return HX
        
    

def House_QR(X):
    A = X.copy()
    n = A.shape[1]
    for i in range(n):
        v, a = house(A[i:,i])
        A[i, i] = a
        A[(i+1):, i] = v[1:]
        A[i:, (i+1):] = Reflection_matrix(A[i:, (i+1):], v)
    
    return A


## (c)

In [4]:
def oper_Q(x, qr):
    y = x.copy()
    m, n = qr.shape
    for i in reversed(range(n)):
        last = m - i
        v = np.zeros(last)
        v[1:] = qr[-(last-1):,i]
        v[0] = np.sqrt(1 - (v[1:]**2).sum())
        y[-last:] = reflection(y[-last:], v)
        
    return y

def oper_Qt(x, qr):
    y = x.copy()
    m, n = qr.shape
    for i in range(n):
        last = m - i
        v = np.zeros(last)
        v[1:] = qr[-(last-1):,i]
        v[0] = np.sqrt(1 - (v[1:]**2).sum())
        y[-last:] = reflection(y[-last:], v)
        
    return y

In [5]:
A = np.random.normal(size = (7, 5))
q0, r0 = np.linalg.qr(A, mode = 'complete')
q1, r1 = GS_QR(A)
qr = House_QR(A)

print(r0)
print(np.triu(qr))

x = np.random.normal(size = (7))
qx0 = q0 @ x
qx2 = oper_Q(x, qr)

print(qx0)
print(qx2)

x = np.random.normal(size = (7))
qx0 = q0.T @ x
qx2 = oper_Qt(x, qr)

print(qx0)
print(qx2)

[[-2.89505268 -1.06263757 -1.35447019 -2.30199241  1.38639329]
 [ 0.         -1.85275273 -0.58023444 -0.49191563 -2.03609884]
 [ 0.          0.         -1.89534093 -0.05940342 -0.66622103]
 [ 0.          0.          0.          2.42555541 -1.40157848]
 [ 0.          0.          0.          0.          2.32813719]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[-2.89505268 -1.06263757 -1.35447019 -2.30199241  1.38639329]
 [ 0.         -1.85275273 -0.58023444 -0.49191563 -2.03609884]
 [ 0.          0.         -1.89534093 -0.05940342 -0.66622103]
 [ 0.          0.          0.          2.42555541 -1.40157848]
 [ 0.          0.          0.          0.          2.32813719]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[-0.71062715 -0.15435791  1.2130751   0.08461637 -0.06396185 -0.38732624
  0.38321013]
[-0.71062715 -0.15435791  1.2

## (d)

## (e) 

In [11]:
def generate_instance(m, n):
    a = np.array([i / (m - 1) for i in range(m)])
    A = np.vander(a, N=n, increasing=True)
    b = np.exp(np.sin(4 * a))
    
    return A, b

def solve1(A, b):
    m, n = A.shape
    qr = House_QR(A)
    c1 = oper_Qt(b, qr)[:n]
    x = solve_triangular(np.triu(qr)[:n,:], c1)
    
    return x


# def solve1_util(qr, b):
#     m, n = qr.shape
#     c1 = oper_Qt(b, qr)[:n]
#     x = solve_triangular(np.triu(qr)[:n,:], c1)
    
    return x
           
def solve2(A, b):
    m, n = A.shape
    qr = House_QR(np.append(A, b[:, None], 1))
    R = np.triu(qr)[:n,:n]
    r = np.triu(qr)[:n,n]
    x = solve_triangular(R, r)
    
    return x



n = 15
m = 100
A, b = generate_instance(m, n)


c = solve1(A, b)
c[-1]
           
c = solve2(A, b)
c[-1]



2006.7868391913557

2006.7868391913557